# Africa's Political Conflicts

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext
import pandas as pd

sc =SparkContext.getOrCreate()
cv = sc.textFile("conflicts.csv")
cv.count()

170579

In [2]:
## fonction
def splitter(lineOfText): 
     words = lineOfText.split(" ") 
     return len(words) 
def aggregate(numWordsLine1, numWordsLineNext): 
     totalWords = numWordsLine1 + numWordsLineNext 
     return totalWords
#Number of words
wordperline = cv.map(splitter)
totwords = wordperline.reduce(aggregate)
totwords

7224133

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark create RDD example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
dataframe = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                load("conflicts.csv")
#display part of the data 
dataframe.show(3)

+---+----------+---+----------+----------+--------------------+--------------------+-------------+--------------------+-------------+-----------+---------------+-------+---------+------------+-----------+-----------------+--------------------+----------+
|_c0|Unnamed: 0|ISO|EVENT_DATE|EVENT_TYPE|      SUB_EVENT_TYPE|              ACTOR1|ASSOC_ACTOR_1|              ACTOR2|ASSOC_ACTOR_2|INTERACTION|         REGION|COUNTRY|   ADMIN1|      ADMIN2|   LOCATION|           SOURCE|               NOTES|FATALITIES|
+---+----------+---+----------+----------+--------------------+--------------------+-------------+--------------------+-------------+-----------+---------------+-------+---------+------------+-----------+-----------------+--------------------+----------+
|  0|         0| 12|2010-12-20|   Battles|         Armed clash|Military Forces o...|         null|Unidentified Arme...|         null|         13|Northern Africa|Algeria|Boumerdes|       Ammal|Ait Dahmane|      TSA Algerie|A militant wa

In [5]:
#remove the first two columns
#display the structure of the data
df = dataframe.drop(dataframe.columns[0],dataframe.columns[1])
df.printSchema()

root
 |-- ISO: integer (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- EVENT_TYPE: string (nullable = true)
 |-- SUB_EVENT_TYPE: string (nullable = true)
 |-- ACTOR1: string (nullable = true)
 |-- ASSOC_ACTOR_1: string (nullable = true)
 |-- ACTOR2: string (nullable = true)
 |-- ASSOC_ACTOR_2: string (nullable = true)
 |-- INTERACTION: integer (nullable = true)
 |-- REGION: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- ADMIN1: string (nullable = true)
 |-- ADMIN2: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- SOURCE: string (nullable = true)
 |-- NOTES: string (nullable = true)
 |-- FATALITIES: integer (nullable = true)



In [6]:
#we filter out the strings containing "Tunisia".
words_filter = cv.filter(lambda x: 'Tunisia' in x)
filtered = words_filter.collect()

print ("Fitered RDD -> %s" % (filtered))

Fitered RDD -> ['133,133,12,2011-02-12,Riots,Mob violence,Police Forces of Algeria (1999-),,Rioters (Algeria),,15,Northern Africa,Algeria,Bejaia,Amizour,Amizour,El Khabar (Tunisia),On 12 February a group of youths clashed with riot police in Amizour Town 50 km to the east of Akbou. The clashes lasted until midnight.,0', '137,137,12,2011-02-12,Riots,Violent demonstration,Police Forces of Algeria (1999-),,Rioters (Algeria),,15,Northern Africa,Algeria,Bejaia,Tinedbar,Sidi Aich,La Tribune (Algeria); El Khabar (Tunisia),In Sidi Aich a group of youths burned tires at an intersection located in downtown disrupting vehicular traffic before traveling to the headquarters of the Security daira for stoning. Police managed to disperse them. The protesters chanted slogans in support of the National Coordination for Change and Democracy (CNCD). They called for the toppling of the regime and for guaranteeing public freedoms.,0', '146,146,12,2011-02-13,Riots,Violent demonstration,Police Forces of Alger

In [7]:
# we form a key value pair and map every string with a value of Tunisia.
words_map = cv.map(lambda x: (x, 'Tunisia'))
mapping = words_map.collect()
print("Key value pair -> %s" % (mapping))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
#we can also check if the RDD is cached or not.
caching = cv.persist().is_cached 
print ( "Words got chached > %s" % (caching))

Words got chached > True


In [12]:
#we can select and show the rows with select and the names of the features
#for example we can show Province compared with Confirmed
df.select('INTERACTION','FATALITIES').show(10)

+-----------+----------+
|INTERACTION|FATALITIES|
+-----------+----------+
|         13|         0|
|         15|         0|
|         12|         2|
|         50|         0|
|         15|         0|
|         15|         0|
|         15|         0|
|         15|         0|
|         13|         1|
|         30|         0|
+-----------+----------+
only showing top 10 rows



In [14]:
#show data sorted by Last Update 
df.sort("FATALITIES",ascending=False).show(2)			

+---+----------+--------------------+--------------+--------------------+-------------+--------------------+--------------------+-----------+--------------+-----------+-------+------+--------+-----------------+--------------------+----------+
|ISO|EVENT_DATE|          EVENT_TYPE|SUB_EVENT_TYPE|              ACTOR1|ASSOC_ACTOR_1|              ACTOR2|       ASSOC_ACTOR_2|INTERACTION|        REGION|    COUNTRY| ADMIN1|ADMIN2|LOCATION|           SOURCE|               NOTES|FATALITIES|
+---+----------+--------------------+--------------+--------------------+-------------+--------------------+--------------------+-----------+--------------+-----------+-------+------+--------+-----------------+--------------------+----------+
|728|2011-08-18|Violence against ...|        Attack|Murle Ethnic Mili...|         null|Civilians (South ...|                null|         47|Eastern Africa|South Sudan|Jonglei|  Uror|   Pieri|Small Arms Survey|Clashes between c...|       750|
|728|2011-08-18|Violence aga

In [16]:
# Countries With no deaths in 2011
sqlContext = SQLContext(sc)
df.registerTempTable("conflicts")	#save the data as a table	
#display Country last updated at 2020-12-07 05:26:14 where Deaths are null.
countries_with_no_deaths= sqlContext.sql("SELECT DISTINCT Country FROM conflicts where EVENT_DATE like '%2011%' and FATALITIES=0")
countries_with_no_deaths.show(10)

+-----------------+
|          Country|
+-----------------+
|             Chad|
|          Senegal|
|          Eritrea|
|         Djibouti|
|           Malawi|
|      Ivory Coast|
|           Rwanda|
|            Sudan|
|          Algeria|
|Equatorial Guinea|
+-----------------+
only showing top 10 rows



In [17]:
# Recovered cases statistics 
df.describe('FATALITIES').show()		

+-------+------------------+
|summary|        FATALITIES|
+-------+------------------+
|  count|            170578|
|   mean|1.7503019146666041|
| stddev| 9.874958100076851|
|    min|                 0|
|    max|               750|
+-------+------------------+



In [19]:
# Recovered cases statistics 
df.describe('INTERACTION').show()		

+-------+------------------+
|summary|       INTERACTION|
+-------+------------------+
|  count|            170578|
|   mean|34.586605541160054|
| stddev|19.075240970846902|
|    min|                10|
|    max|                88|
+-------+------------------+



In [28]:
df.filter(df.EVENT_DATE.contains("2008")).count()

0

In [29]:
df.filter(df.EVENT_DATE.contains("2011")).count()

8127

In [30]:
df.filter(df.EVENT_DATE.contains("2014")).count()

16547

In [31]:
df.filter(df.EVENT_DATE.contains("2018")).count()

19460